In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")
from regtabletotext import prettify_result

In [ ]:
%%time
Cret = pd.read_csv('Data/Cret.csv', index_col=0)
Lret = pd.read_csv('Data/Lret.csv', index_col=0)
RetDiff = pd.read_csv('Data/RetDiff.csv', index_col=0)
MktCap = pd.read_csv('Data/MktCap.csv', index_col=0)
MktCap = np.log(MktCap)

## 新增
liquidity = pd.read_csv('Data/Liquidity.csv', index_col=0)
amount = pd.read_csv('Data/Amount.csv', index_col=0)


Cret.index = pd.to_datetime(Cret.index)
Lret.index = pd.to_datetime(Lret.index)
RetDiff.index = pd.to_datetime(RetDiff.index)
MktCap.index = pd.to_datetime(MktCap.index)
## 新增
liquidity.index = pd.to_datetime(liquidity.index)
amount.index = pd.to_datetime(amount.index)


Cret = Cret.loc['2012/02':]
Lret = Lret.loc['2012/02':]
RetDiff = RetDiff.loc['2012/02':]
MktCap = MktCap.loc['2012/02':]
## 新增
liquidity = liquidity.loc['2012/02':]
amount = amount.loc['2012/02':]

CPU times: total: 1.67 s
Wall time: 1.71 s


In [ ]:
%%time
# -------------------------- Data Preparation for Cross-Sectional Analysis --------------------------
# Core logic: Align lagged predictors (t) with next-month return (t+1) for panel data construction

# 1. Extract lagged predictor variables (time t) and stack into multi-index (Date, Code)
# Stack converts wide-format (Date×Code) DataFrame to long-format Series with (Date, Code) index
# set x (lagged variable)
multi_cret = Cret.iloc[:-1].stack()          # Peer group return (t)
multi_lret = Lret.iloc[:-1].stack()          # Stock's lagged return (t)
multi_retdiff = RetDiff[:-1].stack()         # Return difference (PRG proxy, t)
multi_mktcap = MktCap[:-1].stack()           # log Market capitalization (size proxy, t)
## 新增
multi_liquidity = liquidity[:-1].stack()     # Liquidity measure (e.g., Amihud, t)
multi_amount = amount[:-1].stack()           # dollar Trading amount (liquidity proxy, t)


# 2. Extract next-month return (t+1) and align with lagged predictors (t)
# Shift index to match t (predictors) with t+1 (outcome)
# set y (next-month ret)
ret = Lret.iloc[1:]                          # Stock return (t+1)
ret.index = Lret.iloc[:-1].index             # Align index of t+1 return with t predictors
multi_ret  = ret.stack()                     # Stack to (Date, Code) multi-index


# 3. Standardize variable names and index labels for merging
# Rename Series and set multi-index names (Date, Code) for consistent joining
multi_cret.rename('Cret', inplace=True)
multi_cret.rename_axis(["Date",'Code'],inplace=True)
multi_lret.rename('Lret', inplace=True)
multi_lret.rename_axis(["Date",'Code'],inplace=True)
multi_retdiff.rename('RetDiff', inplace=True)
multi_retdiff.rename_axis(["Date",'Code'],inplace=True)
multi_mktcap.rename('MktCap', inplace=True)
multi_mktcap.rename_axis(["Date",'Code'],inplace=True)
## 新增
multi_liquidity.rename('Liquidity', inplace=True)
multi_liquidity.rename_axis(["Date",'Code'],inplace=True)
multi_amount.rename('Amount', inplace=True)
multi_amount.rename_axis(["Date",'Code'],inplace=True)


multi_ret.rename('Ret_Next', inplace=True)
multi_ret.rename_axis(["Date",'Code'],inplace=True)


# 4. Convert Series to DataFrames for join operation
multi_cret = pd.DataFrame(multi_cret)
multi_lret = pd.DataFrame(multi_lret)
multi_retdiff = pd.DataFrame(multi_retdiff)
multi_mktcap = pd.DataFrame(multi_mktcap)
## 新增
multi_liquidity = pd.DataFrame(multi_liquidity)
multi_amount = pd.DataFrame(multi_amount)

multi_ret  = pd.DataFrame(multi_ret)


# 5. Merge all variables into a single panel DataFrame (Date×Code)
# Final panel data with lagged predictors (t) and next-month return (t+1)
df = (multi_cret
      .join(multi_lret)
      .join(multi_retdiff)
      .join(multi_mktcap)
      .join(multi_liquidity)
      .join(multi_amount)
      .join(multi_ret))

CPU times: total: 516 ms
Wall time: 512 ms


In [4]:
df

Cret      Lret   RetDiff     MktCap  Liquidity  \
Date       Code                                                          
2012-02-29 X600004  0.052158  0.047988 -0.008868  15.867774   0.000598   
           X600005  0.091296  0.059801  0.040118  17.287451   0.000224   
           X600006  0.112744  0.206250 -0.093782  15.859325   0.000651   
           X600007  0.098728  0.097268 -0.005037  16.129344   0.001411   
           X600008  0.103561  0.057471  0.049847  16.312346   0.000277   
...                      ...       ...       ...        ...        ...   
2023-11-30 X300729  0.006134 -0.026639  0.035650  15.258272   0.000133   
           X300730  0.002492  0.140765 -0.137678  14.827270   0.000217   
           X300731  0.030050  0.021530  0.020769  14.801091   0.000120   
           X300732  0.041259  0.004405  0.030125  14.897630   0.000289   
           X300735  0.065603 -0.201775  0.254118  16.718297   0.000015   

                          Amount  Ret_Next  
Date       Code                             
2012-02-29 X600004  2.411051e+07 -0.004431  
           X600005  7.673755e+07 -0.103448  
           X600006  3.741381e+07 -0.145078  
           X600007  7.052508e+06  0.016932  
           X600008  5.980275e+07 -0.108696  
...                          ...       ...  
2023-11-30 X300729  6.362241e+07  0.256842  
           X300730  1.632176e+08 -0.029957  
           X300731  2.536676e+08 -0.060090  
           X300732  1.994576e+07 -0.031798  
           X300735  2.125055e+09 -0.030877  

[231771 rows x 7 columns]

### double sort - hard code of variable name

In [59]:
def conditional_double_sort(df):
    """
    Conditional double sorting for panel data with multi-index (Date, Code)
    Input: 
        df: Multi-index DataFrame (level0=Date, level1=Code) with columns: Liquidity, RetDiff, Ret_Next
    Output: 
        1. PRG effect summary table (cross-tab of liquidity groups vs PRG quintiles)
        2. Detailed portfolio return table (mean return, sample count, Newey-West t-stat)
        3. Formatted PRG effect table with t-statistics
    """
    # -------------------------- Step 1: Monthly double sorting --------------------------
    def monthly_sort(group):
        """Process single-month data (sub-DataFrame grouped by Date)"""
        # Substep 1: Split into 2 liquidity groups by monthly median
        # Note: Amihud liquidity (higher value = lower liquidity) vs trading volume (higher = higher liquidity)
        liquidity_median = group['Liquidity'].median()
        group['Liquidity_Group'] = np.where(
            group['Liquidity'] > liquidity_median, 
            'High_Liquidity',  # Label for liquidity > median (verify indicator direction!)
            'Low_Liquidity'    # Label for liquidity < median
        )
        # Critical correction: 
        # - For Amihud (reverse indicator): swap labels to 'Low_Liquidity' (value > median) / 'High_Liquidity'
        # - For trading volume (positive indicator): keep current label logic

        # Substep 2: Sort RetDiff into 5 quintiles within each liquidity group
        # PRG1 (lowest RetDiff) to PRG5 (highest RetDiff) per Chen et al. (2019)
        for liq_group in ['Low_Liquidity', 'High_Liquidity']:
            sub_group_mask = group['Liquidity_Group'] == liq_group
            sub_group = group[sub_group_mask]
            # Qcut ensures equal sample size across quintiles (drop duplicates to avoid errors)
            group.loc[sub_group_mask, 'RetDiff_Group'] = pd.qcut(
                sub_group['RetDiff'], 
                q=5,  
                labels=['PRG1', 'PRG2', 'PRG3', 'PRG4', 'PRG5'],
                duplicates='drop'  
            )
        return group

    # Execute monthly sorting (group by Date index level 0)
    df_sorted = df.groupby(level=0).apply(monthly_sort)

    # -------------------------- Step 2: Calculate portfolio returns --------------------------
    # Aggregate by liquidity group + PRG quintile: mean return, sample count, simplified Newey-West t-stat
    group_result = df_sorted.groupby(['Liquidity_Group', 'RetDiff_Group'])['Ret_Next'].agg(
        mean_return='mean',  # Monthly average return (core metric)
        count='count',       # Sample count per group (check sorting balance)
        # Simplified Newey-West t-stat (full version see note below)
        nw_t=lambda x: np.mean(x) / (np.std(x, ddof=1) / np.sqrt(len(x)))  
    ).round(6)  # Round to 6 decimals to avoid redundant precision

    # -------------------------- Step 3: Compute PRG effect (PRG5-PRG1) --------------------------
    # Reshape to cross-tab: rows=liquidity groups, columns=PRG quintiles
    prg_effect = group_result['mean_return'].unstack()
    # PRG effect = return difference between highest and lowest RetDiff quintiles
    prg_effect['PRG5-PRG1'] = prg_effect['PRG5'] - prg_effect['PRG1']

    # -------------------------- Step 4: Format output table --------------------------
    # Add t-statistics to PRG effect table (format: return (t-stat))
    prg_t_values = group_result['nw_t'].unstack()
    prg_effect_with_t = prg_effect.copy()
    
    # Format individual quintile returns with t-stat
    for col in prg_effect.columns:
        if col != 'PRG5-PRG1':
            prg_effect_with_t[col] = prg_effect[col].astype(str) + ' (' + prg_t_values[col].round(2).astype(str) + ')'
    
    # Simplified t-stat for PRG5-PRG1 (difference of individual t-stats)
    prg5_t = group_result.loc[(slice(None), 'PRG5'), 'nw_t'].values
    prg1_t = group_result.loc[(slice(None), 'PRG1'), 'nw_t'].values
    prg_effect_with_t['PRG5-PRG1'] = prg_effect['PRG5-PRG1'].astype(str) + ' (' + np.round((prg5_t - prg1_t), 2).astype(str) + ')'

    return prg_effect, group_result, prg_effect_with_t

In [60]:
# 执行双重排序
prg_effect_table, detailed_result, prg_effect_with_t = conditional_double_sort(df)

# 输出核心结果
print("=== 1. PRG效应汇总表（平均月度收益，对标Chen et al. 2019 Table 8） ===")
print(prg_effect_table)
print("\n=== 2. PRG效应表（含Newey-West t值，用于论文表格） ===")
print(prg_effect_with_t)
# print("\n=== 3. 详细分组信息（样本数验证） ===")
# print(detailed_result[['mean_return', 'count']])

=== 1. PRG效应汇总表（平均月度收益，对标Chen et al. 2019 Table 8） ===
RetDiff_Group        PRG1      PRG2      PRG3      PRG4      PRG5  PRG5-PRG1
Liquidity_Group                                                             
High_Liquidity   0.006539  0.017578  0.019354  0.018312  0.016962   0.010423
Low_Liquidity    0.000099  0.005437  0.007304  0.007053  0.005824   0.005725

=== 2. PRG效应表（含Newey-West t值，用于论文表格） ===
RetDiff_Group               PRG1              PRG2              PRG3  \
Liquidity_Group                                                        
High_Liquidity   0.006539 (5.14)  0.017578 (17.19)  0.019354 (20.12)   
Low_Liquidity     9.9e-05 (0.08)   0.005437 (5.68)   0.007304 (7.66)   

RetDiff_Group                PRG4              PRG5  \
Liquidity_Group                                       
High_Liquidity   0.018312 (19.16)  0.016962 (15.96)   
Low_Liquidity     0.007053 (7.48)   0.005824 (5.82)   

RetDiff_Group                       PRG5-PRG1  
Liquidity_Group                      

Typo: high Amihud value represent low liquidity

### double sort: free param for variable name

#### illiquidity

In [62]:
def conditional_double_sort_simple(
    df,
    first_sort_col: str,
    second_sort_col: str,
    return_col: str = 'Ret_Next',
    first_sort_n: int = 2,
    second_sort_n: int = 5,
    first_sort_labels: list = None,
    second_sort_labels: list = None,
    reverse_first_sort: bool = True
):
    """
    Simplified conditional double sorting (parameterized variable names, original calculation logic retained)
    Inputs:
        df: Multi-index DataFrame (level0=Date, level1=Code)
        first_sort_col: Column name for first sorting variable (e.g., 'Liquidity')
        second_sort_col: Column name for second sorting variable (e.g., 'RetDiff')
        return_col: Column name for portfolio return (default: 'Ret_Next')
        first_sort_n: Number of groups for first variable (default: 2)
        second_sort_n: Number of groups for second variable (default: 5)
        first_sort_labels: Custom labels for first sorting groups (auto-generated if None)
        second_sort_labels: Custom labels for second sorting groups (auto-generated if None)
        reverse_first_sort: Whether to reverse first variable labeling (True for reverse indicators like Amihud)
    Outputs:
        1. Effect summary table (cross-tab of groups with return difference column)
        2. Detailed group stats (mean return, sample count, simplified Newey-West t-stat)
        3. Formatted table with returns and t-stats (format: return (t-stat))
    """
    # Initialize group labels (auto-generate if custom labels not provided)
    if first_sort_labels is None:
        first_sort_labels = [f'{first_sort_col}_Group{i+1}' for i in range(first_sort_n)]
    if second_sort_labels is None:
        second_sort_labels = [f'{second_sort_col}_Group{i+1}' for i in range(second_sort_n)]

    # Monthly sorting logic (only variable names parameterized, core logic unchanged)
    def monthly_sort(group):
        # Calculate median for first sorting variable (monthly cross-section)
        first_median = group[first_sort_col].median()
        # Assign first group labels (reverse logic for inverse indicators e.g., Amihud liquidity)
        if reverse_first_sort:
            group['First_Group'] = np.where(
                group[first_sort_col] > first_median,
                first_sort_labels[0],  # Higher value → first label (e.g., Low_Liquidity for Amihud)
                first_sort_labels[1]   # Lower value → second label (e.g., High_Liquidity for Amihud)
            )
        else:
            group['First_Group'] = np.where(
                group[first_sort_col] > first_median,
                first_sort_labels[1],  # Higher value → second label (e.g., High_Size for market cap)
                first_sort_labels[0]   # Lower value → first label (e.g., Low_Size for market cap)
            )

        # Sort second variable into quantiles within each first group
        for first_group in first_sort_labels:
            sub_mask = group['First_Group'] == first_group
            sub_group = group[sub_mask]
            # Qcut for equal-sized groups (drop duplicates to avoid binning errors)
            group.loc[sub_mask, 'Second_Group'] = pd.qcut(
                sub_group[second_sort_col],
                q=second_sort_n,
                labels=second_sort_labels,
                duplicates='drop'
            )
        return group

    # Execute monthly sorting (group by Date index level 0)
    df_sorted = df.groupby(level=0).apply(monthly_sort)

    # Aggregate stats by double-sorted groups (original full-sample logic retained)
    group_result = df_sorted.groupby(['First_Group', 'Second_Group'])[return_col].agg(
        mean_return='mean',          # Full-sample mean return per group
        count='count',               # Sample count per group (check sorting balance)
        nw_t=lambda x: np.mean(x) / (np.std(x, ddof=1) / np.sqrt(len(x)))  # Simplified Newey-West t-stat
    ).round(6)  # Round to 6 decimals to avoid redundant precision

    # Calculate effect (difference between last and first second-sort groups)
    effect_summary = group_result['mean_return'].unstack()
    effect_name = f'{second_sort_labels[-1]}-{second_sort_labels[0]}'  # e.g., PRG5-PRG1
    effect_summary[effect_name] = effect_summary[second_sort_labels[-1]] - effect_summary[second_sort_labels[0]]

    # Format table with returns and rounded t-statistics (original formatting logic)
    formatted_summary = effect_summary.copy()
    t_table = group_result['nw_t'].unstack()
    # Format individual second-sort group returns with t-stats
    for col in second_sort_labels:
        formatted_summary[col] = effect_summary[col].astype(str) + ' (' + t_table[col].round(2).astype(str) + ')'
    
    # Calculate simplified t-stat for effect (difference of individual group t-stats)
    high_t = group_result.loc[(slice(None), second_sort_labels[-1]), 'nw_t'].values
    low_t = group_result.loc[(slice(None), second_sort_labels[0]), 'nw_t'].values
    formatted_summary[effect_name] = effect_summary[effect_name].astype(str) + ' (' + np.round((high_t - low_t), 2).astype(str) + ')'

    # Set index/column names for readability
    effect_summary.index.name = first_sort_col
    effect_summary.columns.name = second_sort_col
    formatted_summary.index.name = first_sort_col
    formatted_summary.columns.name = second_sort_col

    return effect_summary, group_result, formatted_summary

值越大说明越不liquid

In [63]:
# 调用（参数和之前一致）
prg_effect, detailed_result, formatted_prg_effect = conditional_double_sort_simple(
    df=df,
    first_sort_col='Liquidity',
    second_sort_col='RetDiff',
    return_col='Ret_Next',
    first_sort_n=2,
    second_sort_n=5,
    first_sort_labels=['Low_Liquidity', 'High_Liquidity'],
    second_sort_labels=['PRG1', 'PRG2', 'PRG3', 'PRG4', 'PRG5'],
    reverse_first_sort=True
)

# 输出
print("=== 1. PRG效应汇总表（和原代码结果一致） ===")
print(prg_effect)
print("\n=== 2. PRG效应表（含t值） ===")
print(formatted_prg_effect)

=== 1. PRG效应汇总表（和原代码结果一致） ===
RetDiff             PRG1      PRG2      PRG3      PRG4      PRG5  PRG5-PRG1
Liquidity                                                                  
High_Liquidity  0.000099  0.005437  0.007304  0.007053  0.005824   0.005725
Low_Liquidity   0.006539  0.017578  0.019354  0.018312  0.016962   0.010423

=== 2. PRG效应表（含t值） ===
RetDiff                    PRG1              PRG2              PRG3  \
Liquidity                                                             
High_Liquidity   9.9e-05 (0.08)   0.005437 (5.68)   0.007304 (7.66)   
Low_Liquidity   0.006539 (5.14)  0.017578 (17.19)  0.019354 (20.12)   

RetDiff                     PRG4              PRG5  \
Liquidity                                            
High_Liquidity   0.007053 (7.48)   0.005824 (5.82)   
Low_Liquidity   0.018312 (19.16)  0.016962 (15.96)   

RetDiff                            PRG5-PRG1  
Liquidity                                     
High_Liquidity               0.005725 (5.74)  

In [8]:
# 调用（参数和之前一致）
prg_effect, detailed_result, formatted_prg_effect = conditional_double_sort_simple(
    df=df,
    first_sort_col='Liquidity',
    second_sort_col='Cret',
    return_col='Ret_Next',
    first_sort_n=2,
    second_sort_n=5,
    first_sort_labels=['Low_Liquidity', 'High_Liquidity'],
    second_sort_labels=['Cret1', 'Cret2', 'Cret3', 'Cret4', 'Cret5'],
    reverse_first_sort=True
)

# 输出
print("=== 1. Cret效应汇总表（和原代码结果一致） ===")
print(prg_effect)
print("\n=== 2. Cret效应表（含t值） ===")
print(formatted_prg_effect)

=== 1. Cret效应汇总表（和原代码结果一致） ===
Cret               Cret1     Cret2     Cret3     Cret4     Cret5  Cret5-Cret1
Liquidity                                                                    
High_Liquidity  0.002311  0.003591  0.005066  0.005883  0.004937     0.002626
Low_Liquidity   0.011774  0.013448  0.015474  0.014882  0.013582     0.001808

=== 2. Cret效应表（含t值） ===
Cret                       Cret1             Cret2             Cret3  \
Liquidity                                                              
High_Liquidity   0.002311 (2.83)   0.003591 (4.32)   0.005066 (5.95)   
Low_Liquidity   0.011774 (14.68)  0.013448 (15.81)  0.015474 (17.36)   

Cret                       Cret4             Cret5  \
Liquidity                                            
High_Liquidity   0.005883 (6.62)   0.004937 (5.46)   
Low_Liquidity   0.014882 (16.76)  0.013582 (15.45)   

Cret                             Cret5-Cret1  
Liquidity                                     
High_Liquidity  0.00262600000000

In [9]:
# 调用（参数和之前一致）
prg_effect, detailed_result, formatted_prg_effect = conditional_double_sort_simple(
    df=df,
    first_sort_col='Liquidity',
    second_sort_col='Lret',
    return_col='Ret_Next',
    first_sort_n=2,
    second_sort_n=5,
    first_sort_labels=['Low_Liquidity', 'High_Liquidity'],
    second_sort_labels=['Lret1', 'Lret2', 'Lret3', 'Lret4', 'Lret5'],
    reverse_first_sort=True
)

# 输出
print("=== 1. Lret效应汇总表（和原代码结果一致） ===")
print(prg_effect)
print("\n=== 2. Lret效应表（含t值） ===")
print(formatted_prg_effect)

=== 1. Lret效应汇总表（和原代码结果一致） ===
Lret               Lret1     Lret2     Lret3     Lret4     Lret5  Lret5-Lret1
Liquidity                                                                    
High_Liquidity  0.004917  0.005667  0.006599  0.005148 -0.000072    -0.004989
Low_Liquidity   0.013382  0.016716  0.017810  0.016514  0.003877    -0.009505

=== 2. Lret效应表（含t值） ===
Lret                       Lret1            Lret2            Lret3  \
Liquidity                                                            
High_Liquidity   0.004917 (5.88)  0.005667 (7.31)  0.006599 (8.58)   
Low_Liquidity   0.013382 (15.69)  0.016716 (21.7)  0.01781 (22.83)   

Lret                       Lret4             Lret5         Lret5-Lret1  
Liquidity                                                               
High_Liquidity   0.005148 (6.07)  -7.2e-05 (-0.07)   -0.004989 (-5.95)  
Low_Liquidity   0.016514 (20.41)   0.003877 (3.75)  -0.009505 (-11.94)  


#### amount

In [10]:
# 调用（参数和之前一致）
prg_effect, detailed_result, formatted_prg_effect = conditional_double_sort_simple(
    df=df,
    first_sort_col='Amount',
    second_sort_col='RetDiff',
    return_col='Ret_Next',
    first_sort_n=2,
    second_sort_n=5,
    first_sort_labels=['Low_Amount', 'High_Amount'],
    second_sort_labels=['PRG1', 'PRG2', 'PRG3', 'PRG4', 'PRG5'],
    reverse_first_sort=False
)

# 输出
print("=== 1. PRG效应汇总表（和原代码结果一致） ===")
print(prg_effect)
print("\n=== 2. PRG效应表（含t值） ===")
print(formatted_prg_effect)

=== 1. PRG效应汇总表（和原代码结果一致） ===
RetDiff          PRG1      PRG2      PRG3      PRG4      PRG5  PRG5-PRG1
Amount                                                                  
High_Amount -0.002460  0.004334  0.007678  0.006313  0.005280   0.007740
Low_Amount   0.013682  0.016658  0.017758  0.017207  0.016393   0.002711

=== 2. PRG效应表（含t值） ===
RetDiff                  PRG1              PRG2             PRG3  \
Amount                                                             
High_Amount  -0.00246 (-1.88)   0.004334 (4.08)  0.007678 (7.63)   
Low_Amount   0.013682 (12.52)  0.016658 (17.97)  0.017758 (19.2)   

RetDiff                  PRG4             PRG5                     PRG5-PRG1  
Amount                                                                        
High_Amount   0.006313 (6.41)   0.00528 (5.07)                0.00774 (6.95)  
Low_Amount   0.017207 (18.53)  0.016393 (16.4)  0.0027110000000000016 (3.89)  


### single sort

#### amount, CSMAR个股衍生指标 dollar trading volume

In [45]:
data_for_sorts = df.reset_index()
data_for_sorts['MktCap'] = np.exp(data_for_sorts['MktCap'])
data_for_sorts = data_for_sorts.dropna(subset=['RetDiff','Amount', 'MktCap'])
data_for_sorts.head()

,Date,Code,Cret,Lret,RetDiff,MktCap,Liquidity,Amount,Ret_Next
0,2012-02-29,X600004,0.052158,0.047988,-0.008868,7785500.00,0.000598,2.411051e+07,-0.004431
1,2012-02-29,X600005,0.091296,0.059801,0.040118,32199157.64,0.000224,7.673755e+07,-0.103448
2,2012-02-29,X600006,0.112744,0.206250,-0.093782,7720000.00,0.000651,3.741381e+07,-0.145078
3,2012-02-29,X600007,0.098728,0.097268,-0.005037,10113116.64,0.001411,7.052508e+06,0.016932
4,2012-02-29,X600008,0.103561,0.057471,0.049847,12144000.00,0.000277,5.980275e+07,-0.108696


##### EW

In [ ]:
%%time
# -------------------------- Portfolio Analysis (Trading Amount Sort) --------------------------
# Core logic: Sort stocks by trading amount (liquidity proxy) into 5 quintiles, calculate long-short return & significance

# Step 1: Monthly sort stocks into 5 liquidity quintiles by trading amount (Amount)
# qcut ensures equal-sized groups (0-20%: low, 20-40%: 2, ..., 80-100%: high)
Liquidity_portfolios = (data_for_sorts
                   .groupby("Date")  # Monthly cross-sectional sort
                   .apply(lambda x: (x.assign(
                       portfolio=pd.qcut(
                           x["Amount"],  # Sort by trading amount (liquidity proxy)
                           q=[0, 0.2,0.4,0.6,0.8, 1], 
                           labels=["low","2","3","4","high"]))  # Labels for liquidity groups
                    )
                )
                .reset_index(drop=True)
                .groupby(["portfolio","Date"])  # Aggregate by portfolio + months
                .apply(lambda x: np.average(x["Ret_Next"]))  # Equal-weighted next-month return
                .reset_index(name="ret")  # Rename aggregated return column
                )


# Step 2: Construct long-short portfolio (long high liquidity, short low liquidity)
Liquidity_longshort = (Liquidity_portfolios
    .pivot_table(index="Date", columns="portfolio", values="ret")  # Reshape to Date×Portfolio
    .reset_index()
    .assign(long_short=lambda x: x["high"]-x["low"])  # Long-short return = high liquidity - low liquidity
    )


# Step 3: Test long-short return significance with HAC-adjusted OLS (Newey-West, maxlags=6)
# Cov_type="HAC" with maxlags=6 for serial correlation adjustment
model_fit = (sm.OLS.from_formula(
    formula="long_short ~ 1",  # Intercept-only regression (test if mean return ≠ 0)
    data=Liquidity_longshort)
    .fit(cov_type="HAC", cov_kwds={"maxlags": 6})  # HAC adjustment for time-series correlation
)


# Step 4: Output formatted regression results (suppress raw summary, use custom prettify function)
# model_fit.summary()
prettify_result(model_fit)

OLS Model:
long_short ~ 1

Coefficients:
           Estimate  Std. Error  t-Statistic  p-Value
Intercept     -0.02       0.004       -4.708      0.0

Summary statistics:
- Number of observations: 142
- R-squared: 0.000, Adjusted R-squared: 0.000
- F-statistic not available

CPU times: total: 391 ms
Wall time: 378 ms


##### VW

In [47]:
%%time
Liquidity_portfolios = (data_for_sorts
                   .groupby("Date")
                   .apply(lambda x: (x.assign(
                       portfolio=pd.qcut(
                           x["Amount"],q=[0, 0.2,0.4,0.6,0.8, 1], labels=["low","2","3","4","high"]))
                    )
                )
                .reset_index(drop=True)
                .groupby(["portfolio","Date"])
                .apply(lambda x: np.average(x["Ret_Next"], weights=x['MktCap']))
                .reset_index(name="ret")
                )


Liquidity_longshort = (Liquidity_portfolios
    .pivot_table(index="Date", columns="portfolio", values="ret")
    .reset_index()
    .assign(long_short=lambda x: x["high"]-x["low"])
    )


model_fit = (sm.OLS.from_formula(
    formula="long_short ~ 1",
    data=Liquidity_longshort)
    .fit(cov_type="HAC", cov_kwds={"maxlags": 6})
)

# model_fit.summary()
prettify_result(model_fit)

OLS Model:
long_short ~ 1

Coefficients:
           Estimate  Std. Error  t-Statistic  p-Value
Intercept    -0.015       0.008       -1.879     0.06

Summary statistics:
- Number of observations: 37
- R-squared: 0.000, Adjusted R-squared: 0.000
- F-statistic not available

CPU times: total: 453 ms
Wall time: 450 ms


#### RetDiff

In [54]:
data_for_sorts = df.reset_index()
data_for_sorts['MktCap'] = np.exp(data_for_sorts['MktCap'])
data_for_sorts = data_for_sorts.dropna(subset=['RetDiff','Ret_Next', 'MktCap'])
data_for_sorts.head()

,Date,Code,Cret,Lret,RetDiff,MktCap,Liquidity,Amount,Ret_Next
0,2012-02-29,X600004,0.052158,0.047988,-0.008868,7785500.00,0.000598,2.411051e+07,-0.004431
1,2012-02-29,X600005,0.091296,0.059801,0.040118,32199157.64,0.000224,7.673755e+07,-0.103448
2,2012-02-29,X600006,0.112744,0.206250,-0.093782,7720000.00,0.000651,3.741381e+07,-0.145078
3,2012-02-29,X600007,0.098728,0.097268,-0.005037,10113116.64,0.001411,7.052508e+06,0.016932
4,2012-02-29,X600008,0.103561,0.057471,0.049847,12144000.00,0.000277,5.980275e+07,-0.108696


##### EW

In [51]:
%%time
X_portfolios = (data_for_sorts
                   .groupby("Date")
                   .apply(lambda x: (x.assign(
                       portfolio=pd.qcut(
                           x["RetDiff"],
                           q=[0, 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1], 
                           labels=["low","2","3","4","5","6","7","8","9", "high"]))
                    )
                )
                .reset_index(drop=True)
                .groupby(["portfolio","Date"])
                .apply(lambda x: np.average(x["Ret_Next"]))
                .reset_index(name="ret")
                )


X_longshort = (X_portfolios
    .pivot_table(index="Date", columns="portfolio", values="ret")
    .reset_index()
    .assign(long_short=lambda x: x["high"]-x["low"])
    )


model_fit = (sm.OLS.from_formula(
    formula="long_short ~ 1",
    data=X_longshort)
    .fit(cov_type="HAC", cov_kwds={"maxlags": 6})
)

# model_fit.summary()
prettify_result(model_fit)

OLS Model:
long_short ~ 1

Coefficients:
           Estimate  Std. Error  t-Statistic  p-Value
Intercept     0.013       0.003        3.765      0.0

Summary statistics:
- Number of observations: 142
- R-squared: -0.000, Adjusted R-squared: -0.000
- F-statistic not available

CPU times: total: 469 ms
Wall time: 476 ms


##### VW

In [55]:
%%time
X_portfolios = (data_for_sorts
                   .groupby("Date")
                   .apply(lambda x: (x.assign(
                       portfolio=pd.qcut(
                           x["RetDiff"],
                           q=[0, 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1], 
                           labels=["low","2","3","4","5","6","7","8","9", "high"]))
                    )
                )
                .reset_index(drop=True)
                .groupby(["portfolio","Date"])
                .apply(lambda x: np.average(x["Ret_Next"], weights=x["MktCap"]))
                .reset_index(name="ret")
                )


X_longshort = (X_portfolios
    .pivot_table(index="Date", columns="portfolio", values="ret")
    .reset_index()
    .assign(long_short=lambda x: x["high"]-x["low"])
    )


model_fit = (sm.OLS.from_formula(
    formula="long_short ~ 1",
    data=X_longshort)
    .fit(cov_type="HAC", cov_kwds={"maxlags": 6})
)

# model_fit.summary()
prettify_result(model_fit)

OLS Model:
long_short ~ 1

Coefficients:
           Estimate  Std. Error  t-Statistic  p-Value
Intercept     0.007       0.004        1.729    0.084

Summary statistics:
- Number of observations: 142
- R-squared: 0.000, Adjusted R-squared: 0.000
- F-statistic not available

CPU times: total: 469 ms
Wall time: 474 ms
